<img src="img/profile_manoelgadi.png" width=100 height=100 align="right">

Author: Prof. Manoel Gadi

Contact: mfalonso@faculty.ie.edu

Teaching Web: http://mfalonso.pythonanywhere.com

Last revision: 15/February/2024

# Objective of the session: 
## Apply SVM for Regression also known as SVR to Company Ratings development

## import the necessary libraries required 

In [4]:
import pandas as pd

### Reading data

In [6]:
df = pd.read_excel("data/TurnOverEstimation.xlsx")

In [7]:
df.head(1).T

0
nif_fical_number_id             A08620668
company_name                     Ancau SA
CNAE                                 1013
p10000_TotalAssets_h0          2041.30776
p10000_TotalAssets_h1          1990.47957
p10000_TotalAssets_h2          1981.58111
p20000_OwnCapital_h0            797.34371
p20000_OwnCapital_h1            789.96194
p20000_OwnCapital_h2            776.50737
p31200_ShortTermDebt_h0           218.592
p31200_ShortTermDebt_h1          240.1886
p31200_ShortTermDebt_h2         365.86712
p32300_LongTermDebt_h0          234.78828
p32300_LongTermDebt_h1          164.50958
p32300_LongTermDebt_h2           62.36545
p40100_40500_SalesTurnover_h0  6635.72582
p40100_40500_SalesTurnover_h1  6556.58451
p40100_40500_SalesTurnover_h2  6489.40907
p40800_Amortization_h0          -52.55223
p40800_Amortization_h1          -54.83961
p40800_Amortization_h2          -47.63204
p49100_Profit_h0                 22.28278
p49100_Profit_h1                 25.63743
p49100_Profit_h2                 29.41425

### Checking for missing values in the data set

In [9]:
df.isnull().sum()

nif_fical_number_id              0
company_name                     0
CNAE                             0
p10000_TotalAssets_h0            0
p10000_TotalAssets_h1            0
p10000_TotalAssets_h2            0
p20000_OwnCapital_h0             0
p20000_OwnCapital_h1             0
p20000_OwnCapital_h2             0
p31200_ShortTermDebt_h0          0
p31200_ShortTermDebt_h1          0
p31200_ShortTermDebt_h2          0
p32300_LongTermDebt_h0           0
p32300_LongTermDebt_h1           0
p32300_LongTermDebt_h2           0
p40100_40500_SalesTurnover_h0    0
p40100_40500_SalesTurnover_h1    0
p40100_40500_SalesTurnover_h2    0
p40800_Amortization_h0           0
p40800_Amortization_h1           0
p40800_Amortization_h2           0
p49100_Profit_h0                 0
p49100_Profit_h1                 0
p49100_Profit_h2                 0
dtype: int64

## Some explanation of the data
* _h0, _h1, _h2
* _h0: history 0, here h0 means the year 2017 (historia 0, aquí h0 significa el año 2017)
* _h1: history -1, here h1 means the year 2016 (historia -1, aquí h1 significa el año 2016)
* _h2: history -2, here h2 means the year 2015 (historia -2, aquí h2 significa el año 2015)

* Ebita Margin: Ebitda / Turn over (Ventas)
* p49100: Profit (Resultado del ejercicio)
* p40800: Amortization (Amortización) 
* p40100: Sales Turnover (Ingresos de Explotación)
* p40500: Other sales (Otros Ingresos)

In [11]:
df['ebitda_income_h1'] = (df.p49100_Profit_h1-df.p40800_Amortization_h1)/(df.p40100_40500_SalesTurnover_h1) 
df[['company_name','ebitda_income_h1']].head(5)



company_name  ebitda_income_h1
0            Ancau SA          0.012274
1         Farmunid SL          0.027920
2     Ganados Lito SL          0.018362
3  Dismol Masquefa SL          0.052426
4  Centurion Playa SA          0.319683

* Total Debt / Ebita 
* p31200: Short Term Debt / Deuda a corto plazo
* p32300: Long Term Debt / Deuda a largo plazo
* p49100: Profit (Resultado del ejercicio)
* p40800: Amortization (Amortización) 


In [13]:
df['debt_ebitda_h1'] =(df.p31200_ShortTermDebt_h1 + df.p32300_LongTermDebt_h1) /(df.p49100_Profit_h1-df.p40800_Amortization_h1) 
df[['company_name','debt_ebitda_h1']].head(5)


company_name  debt_ebitda_h1
0            Ancau SA        5.028741
1         Farmunid SL        1.167992
2     Ganados Lito SL        0.000000
3  Dismol Masquefa SL        7.162535
4  Centurion Playa SA        1.676169

* rraa_rrpp: Financial leveraging / apalancamiento financiero 
* p10000: Total Assets / Total activos
* p20000: Own Capital / Patrimonio neto

In [15]:
df['rraa_rrpp_h1'] = (df.p10000_TotalAssets_h1 - df.p20000_OwnCapital_h1) /df.p20000_OwnCapital_h1
df[['company_name','rraa_rrpp_h1']].head(5)

company_name  rraa_rrpp_h1
0            Ancau SA      1.519716
1         Farmunid SL      2.131077
2     Ganados Lito SL      0.136521
3  Dismol Masquefa SL      2.233652
4  Centurion Playa SA      1.083639

* Log of Operating Income



In [17]:
import numpy as np
df['log_operating_income_h1'] = np.log(df.p40100_40500_SalesTurnover_h1)
df[['company_name','log_operating_income_h1']].head(5)

company_name  log_operating_income_h1
0            Ancau SA                 8.788225
1         Farmunid SL                 8.753590
2     Ganados Lito SL                 8.752961
3  Dismol Masquefa SL                 8.681598
4  Centurion Playa SA                 8.813364

## Cleaning data - replacing all NaN and Inf/-Inf with 0

In [19]:
df = df.fillna(0).replace(np.inf, 0).replace(-np.inf, 0)

--- 
## Doing some EDA

Target: p40100_40500_SalesTurnover_h0

In [22]:
variables_to_analyse = ['p10000_TotalAssets_h1', 'p10000_TotalAssets_h2',
       'p20000_OwnCapital_h1', 'p20000_OwnCapital_h2',
       'p31200_ShortTermDebt_h1',
       'p31200_ShortTermDebt_h2',
       'p32300_LongTermDebt_h1', 'p32300_LongTermDebt_h2',
       'p40100_40500_SalesTurnover_h1',
       'p40100_40500_SalesTurnover_h2',
       'p40800_Amortization_h1', 'p40800_Amortization_h2',
       'p49100_Profit_h1', 'p49100_Profit_h2', 'ebitda_income_h1',
       'debt_ebitda_h1', 'rraa_rrpp_h1', 'log_operating_income_h1']

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics

#%matplotlib notebook
%matplotlib inline


In [24]:
type(df)

pandas.core.frame.DataFrame

## Basic Data Analysis or Exploratory Data Analysis

In [26]:
print("The number of rows and colums are {} and also called shape of the matrix".format(df[variables_to_analyse].shape))
print("Columns names are \n {}".format(df[variables_to_analyse].columns))

The number of rows and colums are (1191, 18) and also called shape of the matrix
Columns names are 
 Index(['p10000_TotalAssets_h1', 'p10000_TotalAssets_h2',
       'p20000_OwnCapital_h1', 'p20000_OwnCapital_h2',
       'p31200_ShortTermDebt_h1', 'p31200_ShortTermDebt_h2',
       'p32300_LongTermDebt_h1', 'p32300_LongTermDebt_h2',
       'p40100_40500_SalesTurnover_h1', 'p40100_40500_SalesTurnover_h2',
       'p40800_Amortization_h1', 'p40800_Amortization_h2', 'p49100_Profit_h1',
       'p49100_Profit_h2', 'ebitda_income_h1', 'debt_ebitda_h1',
       'rraa_rrpp_h1', 'log_operating_income_h1'],
      dtype='object')


### display scatter_matrix also

In [28]:
fig = plt.figure()
scatter_matrix(df[variables_to_analyse],figsize =(25,25),alpha=0.9,diagonal="kde",marker="o");

### some histograms

In [30]:
df[variables_to_analyse].hist(figsize=(25,25),bins=50);

In [31]:
hcorr = df[variables_to_analyse].corr()
hcorr.style.background_gradient()

## heatmap using seaborn

In [33]:
#heatmap using seaborn
#set the context for plotting 
sns.set(context="paper",font="monospace")
df_corr_matrix = df[variables_to_analyse].corr()
#set the matplotlib figure
fig, axe = plt.subplots(figsize=(12,8))
#Generate color palettes 
cmap = sns.diverging_palette(220,10,center = "light", as_cmap=True)
#draw the heatmap
sns.heatmap(df_corr_matrix,vmax=1,square =True, cmap=cmap,annot=True );

# NEW CONCEPT: Label encode for categorical feature (CNAE)

In [35]:
labelEncoder = LabelEncoder()
print(df["CNAE"].value_counts())
df["CNAE_encoded"] = labelEncoder.fit_transform(df["CNAE"])
print(df["CNAE_encoded"].value_counts())
print(df.describe())

CNAE
4631    47
5510    35
4110    33
4730    30
6820    27
        ..
3316     1
2825     1
6831     1
3700     1
2811     1
Name: count, Length: 309, dtype: int64
CNAE_encoded
158    47
226    35
126    33
197    30
257    27
       ..
115     1
97      1
258     1
122     1
93      1
Name: count, Length: 309, dtype: int64
              CNAE  p10000_TotalAssets_h0  p10000_TotalAssets_h1  \
count  1191.000000           1.191000e+03            1191.000000   
mean   4484.240134           1.352460e+04           12121.939456   
std    1924.685608           6.530715e+04           39591.176354   
min     113.000000           4.368217e+02              32.451660   
25%    3519.000000           2.756224e+03            2553.204865   
50%    4638.000000           4.580959e+03            4238.401540   
75%    5210.000000           7.491824e+03            7106.563465   
max    9603.000000           1.905343e+06          647169.906460   

       p10000_TotalAssets_h2  p20000_OwnCapital_h0  p20000_O

### Split the dataset into 80% train and 20% test dataset

In [37]:
df.columns

Index(['nif_fical_number_id', 'company_name', 'CNAE', 'p10000_TotalAssets_h0',
       'p10000_TotalAssets_h1', 'p10000_TotalAssets_h2',
       'p20000_OwnCapital_h0', 'p20000_OwnCapital_h1', 'p20000_OwnCapital_h2',
       'p31200_ShortTermDebt_h0', 'p31200_ShortTermDebt_h1',
       'p31200_ShortTermDebt_h2', 'p32300_LongTermDebt_h0',
       'p32300_LongTermDebt_h1', 'p32300_LongTermDebt_h2',
       'p40100_40500_SalesTurnover_h0', 'p40100_40500_SalesTurnover_h1',
       'p40100_40500_SalesTurnover_h2', 'p40800_Amortization_h0',
       'p40800_Amortization_h1', 'p40800_Amortization_h2', 'p49100_Profit_h0',
       'p49100_Profit_h1', 'p49100_Profit_h2', 'ebitda_income_h1',
       'debt_ebitda_h1', 'rraa_rrpp_h1', 'log_operating_income_h1',
       'CNAE_encoded'],
      dtype='object')

## X takes all data from 2016 (h1) and 2015 (h2)

In [39]:
X = df[['p10000_TotalAssets_h1', 'p10000_TotalAssets_h2',
       'p20000_OwnCapital_h1', 'p20000_OwnCapital_h2',
       'p31200_ShortTermDebt_h1',
       'p31200_ShortTermDebt_h2',
       'p32300_LongTermDebt_h1', 'p32300_LongTermDebt_h2',
       'p40100_40500_SalesTurnover_h1',
       'p40100_40500_SalesTurnover_h2',
       'p40800_Amortization_h1', 'p40800_Amortization_h2',
       'p49100_Profit_h1', 'p49100_Profit_h2', 'CNAE_encoded','ebitda_income_h1',
       'debt_ebitda_h1', 'rraa_rrpp_h1', 'log_operating_income_h1']]

In [40]:
y = df['p40100_40500_SalesTurnover_h0']

In [41]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [42]:
X_train.head()

p10000_TotalAssets_h1  p10000_TotalAssets_h2  p20000_OwnCapital_h1  \
224             1055.06381              980.22941             159.15071   
692             4598.32475             3617.19465            2162.88834   
654             2278.70500             1715.81393             458.50454   
428             1556.07458             1232.29409             448.86552   
780             2996.95027             2729.30228             652.62008   

     p20000_OwnCapital_h2  p31200_ShortTermDebt_h1  p31200_ShortTermDebt_h2  \
224             137.62005                  0.00000                  0.00000   
692            1385.59188                  3.24203                 91.22529   
654             376.04792                386.71223                205.79533   
428             415.75289                 28.32521                 30.25375   
780             609.55055                326.19196                165.27888   

     p32300_LongTermDebt_h1  p32300_LongTermDebt_h2  \
224                89.23149               183.97621   
692               573.86544                59.24515   
654               658.65472               546.34967   
428               199.74258                99.85984   
780                 0.00000                 0.00000   

     p40100_40500_SalesTurnover_h1  p40100_40500_SalesTurnover_h2  \
224                     5525.37840                     6449.30544   
692                     7755.16235                     9352.17773   
654                     5454.80146                     4419.52660   
428                     5586.09327                     7559.88840   
780                     5614.04599                     5512.53659   

     p40800_Amortization_h1  p40800_Amortization_h2  p49100_Profit_h1  \
224               -20.00596               -17.58491          35.43014   
692               -35.10460               -33.56363        1150.67313   
654               -62.04427               -57.33793         140.63759   
428                -5.47667                -5.05017          44.27183   
780               -17.27243               -10.86738          68.42320   

     p49100_Profit_h2  CNAE_encoded  ebitda_income_h1  debt_ebitda_h1  \
224          56.73401           147          0.010033        1.609628   
692        1758.71829            39          0.152902        0.486691   
654          95.73391           161          0.037157        5.157674   
428         292.47791           158          0.008906        4.584415   
780          64.53360           194          0.015265        3.806401   

     rraa_rrpp_h1  log_operating_income_h1  
224      5.629338                 8.617107  
692      1.126011                 8.956114  
654      3.969864                 8.604252  
428      2.466683                 8.628035  
780      3.592182                 8.633027

---

# Performing Linear Regression

In most linear regression models, the objective is to minimize the sum of squared errors. Take Ordinary Least Squares (OLS) for example. The objective function for OLS with one predictor (feature) is as follows:

<img src="img/simple_ols.png" align="center">


In [45]:
#initantiate the linear regression
linearRegModel = LinearRegression(n_jobs=-1)
#fit the model to the training data (learn the coefficients)
linearRegModel.fit(X_train,y_train)
#print the intercept and coefficients 
print("Intercept is "+str(linearRegModel.intercept_))
print("coefficients  is "+str(linearRegModel.coef_))

Intercept is 6357.5602612434095
coefficients  is [-1.07512059e-03  1.59356883e-03  6.39965896e-04 -1.13913770e-03
 -3.40800838e-05 -6.01238295e-05 -1.81897535e-04 -5.29729971e-04
 -2.49897891e-03  6.54588930e-03 -1.08661995e-02  9.31371813e-03
  1.94347152e-03 -1.30604939e-03  7.82618578e-02  2.43325155e+01
  1.73611074e-01 -3.62293774e-02  3.46699972e+00]


In [46]:
#predict on the test data
y_pred = linearRegModel.predict(X_test)

In [47]:
print(len(y_pred))
print(len(y_test))
print(y_pred[0:5])
print(y_test[0:5])

239
239
[6454.65371189 6416.22393022 6442.52099355 6412.62628896 6437.9353642 ]
953    6303.14853
247    6547.95216
260    6544.00000
155    6581.25590
566    6433.04858
Name: p40100_40500_SalesTurnover_h0, dtype: float64


In [48]:
test = pd.DataFrame({'Predicted':y_pred,'Actual':y_test})
fig= plt.figure(figsize=(16,8))
test = test.reset_index()
test = test.drop(['index'],axis=1)
plt.plot(test[:50])
plt.legend(['Actual','Predicted'])
sns.jointplot(x='Actual',y='Predicted',data=test,kind='reg',);

### Root Mean Squared Error (RMSE)

In [50]:
print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print(np.sqrt(metrics.mean_squared_error(y_train,linearRegModel.predict(X_train))))

136.05975199456992
149.11951846236897


---

# Performing other regressions

Lasso, Ridge, and ElasticNet are all extensions of the previous simple equation, with an additional penalty parameter.

Bayesian Regression is a type of regularization, which uses distributions instead of fix estimation and the bayesian equation uses combination of other parameters to estimate one.


In [53]:
# MODELING
from sklearn.linear_model import LinearRegression # OLS algorithm
from sklearn.linear_model import Ridge # Ridge algorithm
from sklearn.linear_model import Lasso # Lasso algorithm
from sklearn.linear_model import BayesianRidge # Bayesian algorithm
from sklearn.svm import SVR

# 1. OLS
ols = LinearRegression()
ols.fit(X_train, y_train)
ols_test_yhat = ols.predict(X_test)
ols_train_yhat = ols.predict(X_train)

# 2. Ridge
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train, y_train)
ridge_test_yhat = ridge.predict(X_test)
ridge_train_yhat = ridge.predict(X_train)

# 3. Lasso

lasso = Lasso(alpha = 0.01)
lasso.fit(X_train, y_train)
lasso_test_yhat = lasso.predict(X_test)
lasso_train_yhat = lasso.predict(X_train)

# 4. Bayesian
bayesian = BayesianRidge()
bayesian.fit(X_train, y_train)
#bayesian_yhat = bayesian.predict(X_test)
bayesian_test_yhat = bayesian.predict(X_test)
bayesian_train_yhat = bayesian.predict(X_train)

# 4. SVR
svr = SVR()
svr.fit(X_train, y_train)
#bayesian_yhat = bayesian.predict(X_test)
svr_test_yhat = svr.predict(X_test)
svr_train_yhat = svr.predict(X_train)


D:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.058e+07, tolerance: 2.172e+03
  model = cd_fast.enet_coordinate_descent(


In [54]:
# 1. mean_squared_error
from termcolor import colored as cl # text customization
from sklearn.metrics import explained_variance_score as evs # evaluation metric

print(cl('EXPLAINED VARIANCE SCORE:', attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('OLS model:', attrs = ['bold']))
print(cl('Test - R^2 model is {}'.format(np.sqrt(metrics.mean_squared_error(y_test,ols_test_yhat))), attrs = ['bold']))


print('-------------------------------------------------------------------------------')
print(cl('Ridge model:', attrs = ['bold']))
print(cl('Test - R^2 model is {}'.format(np.sqrt(metrics.mean_squared_error(y_test,ridge_test_yhat))), attrs = ['bold']))


print('-------------------------------------------------------------------------------')
print(cl('Lasso model:', attrs = ['bold']))
print(cl('Test - R^2 model is {}'.format(np.sqrt(metrics.mean_squared_error(y_test,lasso_test_yhat))), attrs = ['bold']))


print('-------------------------------------------------------------------------------')
print(cl('Bayesian model:', attrs = ['bold']))
print(cl('Test - R^2 model is {}'.format(np.sqrt(metrics.mean_squared_error(y_test,bayesian_test_yhat))), attrs = ['bold']))



print('-------------------------------------------------------------------------------')
print(cl('SVR model:', attrs = ['bold']))
print(cl('Test - R^2 model is {}'.format(np.sqrt(metrics.mean_squared_error(y_test, svr_test_yhat))), attrs = ['bold']))


EXPLAINED VARIANCE SCORE:
-------------------------------------------------------------------------------
OLS model:
Test - R^2 model is 136.05975199456992
-------------------------------------------------------------------------------
Ridge model:
Test - R^2 model is 135.52359019714558
-------------------------------------------------------------------------------
Lasso model:
Test - R^2 model is 135.70642849512942
-------------------------------------------------------------------------------
Bayesian model:
Test - R^2 model is 116.59835723179313
-------------------------------------------------------------------------------
SVR model:
Test - R^2 model is 115.46039623094471


---

# Standardize the data
Repeating analysis with standardized data

In [57]:
independent_scaler = StandardScaler()
X_train = independent_scaler.fit_transform(X_train)
X_test = independent_scaler.transform(X_test)

In [58]:
X_train

array([[-0.29606509, -0.34967449, -0.20624391, ..., -0.05006527,
         0.11106577, -0.0242399 ],
       [-0.19204551, -0.25047452, -0.13113454, ..., -0.07249514,
        -0.04776586,  0.66351576],
       [-0.26014263, -0.32200254, -0.19502274, ...,  0.02080447,
         0.05253637, -0.05032033],
       ...,
       [ 0.31982533,  0.31115736,  0.50366824, ..., -0.07135474,
        -0.08205703,  0.22352212],
       [-0.19548584, -0.22876444, -0.14585045, ..., -0.04940703,
        -0.03347205,  0.18183511],
       [-0.25953581, -0.34330297, -0.19143271, ..., -0.06489749,
         0.02356408,  0.4455097 ]])

## Perform Linear Regression with Standardized Data

In [60]:
# MODELING
from sklearn.linear_model import LinearRegression # OLS algorithm
from sklearn.linear_model import Ridge # Ridge algorithm
from sklearn.linear_model import Lasso # Lasso algorithm
from sklearn.linear_model import BayesianRidge # Bayesian algorithm
from sklearn.svm import SVR

# 1. OLS
ols = LinearRegression()
ols.fit(X_train, y_train)
ols_test_yhat = ols.predict(X_test)
ols_train_yhat = ols.predict(X_train)

# 2. Ridge
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train, y_train)
ridge_test_yhat = ridge.predict(X_test)
ridge_train_yhat = ridge.predict(X_train)

# 3. Lasso

lasso = Lasso(alpha = 0.01)
lasso.fit(X_train, y_train)
lasso_test_yhat = lasso.predict(X_test)
lasso_train_yhat = lasso.predict(X_train)

# 4. Bayesian
bayesian = BayesianRidge()
bayesian.fit(X_train, y_train)
#bayesian_yhat = bayesian.predict(X_test)
bayesian_test_yhat = bayesian.predict(X_test)
bayesian_train_yhat = bayesian.predict(X_train)

# 4. SVR
svr = SVR()
svr.fit(X_train, y_train)
#bayesian_yhat = bayesian.predict(X_test)
svr_test_yhat = svr.predict(X_test)
svr_train_yhat = svr.predict(X_train)


D:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.543e+05, tolerance: 2.172e+03
  model = cd_fast.enet_coordinate_descent(


In [61]:
# MODELING
from sklearn.linear_model import LinearRegression # OLS algorithm
from sklearn.linear_model import Ridge # Ridge algorithm
from sklearn.linear_model import Lasso # Lasso algorithm
from sklearn.linear_model import BayesianRidge # Bayesian algorithm
from sklearn.svm import SVR

# 1. OLS
ols = LinearRegression()
ols.fit(X_train, y_train)
ols_test_yhat = ols.predict(X_test)
ols_train_yhat = ols.predict(X_train)

# 2. Ridge
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train, y_train)
ridge_test_yhat = ridge.predict(X_test)
ridge_train_yhat = ridge.predict(X_train)

# 3. Lasso

lasso = Lasso(alpha = 0.01)
lasso.fit(X_train, y_train)
lasso_test_yhat = lasso.predict(X_test)
lasso_train_yhat = lasso.predict(X_train)

# 4. Bayesian
bayesian = BayesianRidge()
bayesian.fit(X_train, y_train)
#bayesian_yhat = bayesian.predict(X_test)
bayesian_test_yhat = bayesian.predict(X_test)
bayesian_train_yhat = bayesian.predict(X_train)

# 4. SVR
svr = SVR()
svr.fit(X_train, y_train)
#bayesian_yhat = bayesian.predict(X_test)
svr_test_yhat = svr.predict(X_test)
svr_train_yhat = svr.predict(X_train)


# 1. mean_squared_error
from termcolor import colored as cl # text customization
from sklearn.metrics import explained_variance_score as evs # evaluation metric

print(cl('EXPLAINED VARIANCE SCORE:', attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('OLS model:', attrs = ['bold']))
print(cl('Test - R^2 model is {}'.format(np.sqrt(metrics.mean_squared_error(y_test,ols_test_yhat))), attrs = ['bold']))


print('-------------------------------------------------------------------------------')
print(cl('Ridge model:', attrs = ['bold']))
print(cl('Test - R^2 model is {}'.format(np.sqrt(metrics.mean_squared_error(y_test,ridge_test_yhat))), attrs = ['bold']))


print('-------------------------------------------------------------------------------')
print(cl('Lasso model:', attrs = ['bold']))
print(cl('Test - R^2 model is {}'.format(np.sqrt(metrics.mean_squared_error(y_test,lasso_test_yhat))), attrs = ['bold']))


print('-------------------------------------------------------------------------------')
print(cl('Bayesian model:', attrs = ['bold']))
print(cl('Test - R^2 model is {}'.format(np.sqrt(metrics.mean_squared_error(y_test,bayesian_test_yhat))), attrs = ['bold']))



print('-------------------------------------------------------------------------------')
print(cl('SVR model:', attrs = ['bold']))
print(cl('Test - R^2 model is {}'.format(np.sqrt(metrics.mean_squared_error(y_test, svr_test_yhat))), attrs = ['bold']))


D:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.543e+05, tolerance: 2.172e+03
  model = cd_fast.enet_coordinate_descent(


EXPLAINED VARIANCE SCORE:
-------------------------------------------------------------------------------
OLS model:
Test - R^2 model is 136.05975199456887
-------------------------------------------------------------------------------
Ridge model:
Test - R^2 model is 136.36520654513455
-------------------------------------------------------------------------------
Lasso model:
Test - R^2 model is 136.44858483849512
-------------------------------------------------------------------------------
Bayesian model:
Test - R^2 model is 118.97731659921234
-------------------------------------------------------------------------------
SVR model:
Test - R^2 model is 115.07306931782168


# Implementing SVR with Different "Penalties" and Kernels
Since SVR doesn't have explicit L1/ElasticNet penalties, we'll demonstrate how to use different C values (for regularization similar to L2) and different kernels. For a truly "no penalty" scenario, you'd conceptually use a very large C value, though this can lead to overfitting.

Claro, aquí tienes la explicación del Kernel RBF en español, más concisa y como si fuera en un cuaderno de notas:

---

## 📝 **Kernel RBF (Radial Basis Function)**

* **¿Qué es?** ➡️ Un "truco" matemático para que los modelos (ej. SVR) manejen datos NO lineales.
* **Problema inicial:** Si tus datos no se pueden separar con una línea recta (ej. círculos concéntricos), un modelo lineal falla.
* **La "Magia" (Kernel Trick):**
    * No transformamos los datos explícitamente a un espacio de mayor dimensión (sería caro).
    * ¡Pero el Kernel RBF *calcula la similitud* como si ya estuvieran en ese espacio!
    * En ese espacio "imaginario" de alta dimensión, ¡los datos a menudo SÍ se pueden separar linealmente!
* **Fórmula (simple):**
    $K(x, x') = \exp(-\gamma ||x - x'||^2)$
    * $||x - x'||^2$: Distancia entre dos puntos ($x, x'$).
    * $\exp(...)$: Convierte la distancia en **similitud**. (Cerca $\approx 1$, Lejos $\approx 0$).
    * $\gamma$ (gamma): Controla el "alcance" o la "influencia" de cada punto.
        * $\gamma$ pequeño ➡️ Influencia amplia, frontera suave.
        * $\gamma$ grande ➡️ Influencia estrecha, frontera compleja (cuidado: overfitting).
* **¿Cómo funciona (intuición)?**
    * Piensa que cada punto de dato crea una "montañita" (campana de Gauss) en un espacio superior.
    * El modelo (ej. SVR) busca una "pared" recta que separe estas montañitas.
    * Esa "pared" recta en el espacio de montañitas se traduce en una frontera *curva* o *compleja* en tu espacio de datos original. **como la distancia de Mahalanobis**
* **Beneficios:**
    * Ideal para relaciones NO lineales.
    * Solo un hiperparámetro clave a ajustar ($\gamma$).

SVR -  parameters: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html


Aquí tienes los tipos de **kernels** que puedes usar en SVR (y otros modelos basados en kernels) explicados en puntos cortos:

---

* **'linear'**:
    * Crea una **línea recta** (o hiperplano) para ajustar los datos.
    * Útil cuando la relación entre tus variables es **lineal**.
    * Fórmula: $K(x, x') = x \cdot x'$.
    * Generalmente **rápido** para entrenar.

* **'poly'** (Polinomial):
    * Crea una **curva** (o superficie polinomial) para ajustar los datos.
    * Útil para relaciones **no lineales** moderadas.
    * Tiene un parámetro `degree` (grado) que define la complejidad de la curva.
    * Puede ser computacionalmente más **costoso** que el lineal y propenso a overfitting con grados altos.

* **'rbf'** (Radial Basis Function / Gaussiano):
    * **Por defecto** en SVR.
    * Crea una frontera de decisión **no lineal** y muy flexible.
    * Funciona midiendo la **similitud** entre puntos usando una "campana de Gauss".
    * Tiene el parámetro `gamma` que controla el "alcance" de la similitud (afecta la complejidad).
    * Excelente para relaciones **complejas y no lineales**.

* **'sigmoid'**:
    * Similar a una **red neuronal de una capa** (con activación sigmoide).
    * También para relaciones **no lineales**.
    * Puede funcionar bien en ciertos casos, pero a menudo es **menos estable** que RBF o lineal en la práctica y puede no ser la primera opción.

* **'precomputed'**:
    * No calcula el kernel, sino que **espera que tú le pases una matriz de kernel** ya calculada.
    * Útil si ya tienes una medida de similitud entre tus puntos o si quieres usar un kernel **personalizado** que no está en la lista estándar.
    * Requiere que le pases la matriz de kernel durante el `.fit()` y `.predict()`.

---

In [63]:
# MODELING
from sklearn.linear_model import LinearRegression # OLS algorithm
from sklearn.linear_model import Ridge # Ridge algorithm
from sklearn.linear_model import Lasso # Lasso algorithm
from sklearn.linear_model import BayesianRidge # Bayesian algorithm
from sklearn.svm import SVR
from sklearn.kernel_approximation import Nystroem # For Nystrom RBF approximation
import numpy as np
from sklearn import metrics # Assuming metrics is imported for mean_squared_error
from termcolor import colored as cl # text customization

# Assuming X_train, y_train, X_test, y_test are already defined from your data splitting

# --- Existing Models (for context and comparison) ---

# 1. OLS
ols = LinearRegression()
ols.fit(X_train, y_train)
ols_test_yhat = ols.predict(X_test)
ols_train_yhat = ols.predict(X_train)

# 2. Ridge
ridge = Ridge(alpha = 0.5)
ridge.fit(X_train, y_train)
ridge_test_yhat = ridge.predict(X_test)
ridge_train_yhat = ridge.predict(X_train)

# 3. Lasso
lasso = Lasso(alpha = 0.01)
lasso.fit(X_train, y_train)
lasso_test_yhat = lasso.predict(X_test)
lasso_train_yhat = lasso.predict(X_train)

# 4. Bayesian
bayesian = BayesianRidge()
bayesian.fit(X_train, y_train)
bayesian_test_yhat = bayesian.predict(X_test)
bayesian_train_yhat = bayesian.predict(X_train)

# --- SVR Models with "Penalties" and Kernels ---

# 5. SVR (Default - RBF kernel, C=1.0)
print(cl('\n--- SVR Models ---', attrs = ['bold']))
svr_default = SVR()
svr_default.fit(X_train, y_train)
svr_default_test_yhat = svr_default.predict(X_test)
svr_default_train_yhat = svr_default.predict(X_train)

# 6. SVR with Linear Kernel (L2-like penalty controlled by C)
# Low C (more regularization, smaller weights)
svr_linear_low_c = SVR(kernel='linear', C=0.1)
svr_linear_low_c.fit(X_train, y_train)
svr_linear_low_c_test_yhat = svr_linear_low_c.predict(X_test)

# High C (less regularization, larger weights, closer to no penalty)
svr_linear_high_c = SVR(kernel='linear', C=100.0)
svr_linear_high_c.fit(X_train, y_train)
svr_linear_high_c_test_yhat = svr_linear_high_c.predict(X_test)

# 7. SVR with RBF Kernel (L2-like penalty controlled by C)
# Low C (more regularization)
svr_rbf_low_c = SVR(kernel='rbf', C=0.1, gamma='scale') # gamma='scale' is a good default
svr_rbf_low_c.fit(X_train, y_train)
svr_rbf_low_c_test_yhat = svr_rbf_low_c.predict(X_test)

# High C (less regularization)
svr_rbf_high_c = SVR(kernel='rbf', C=100.0, gamma='scale')
svr_rbf_high_c.fit(X_train, y_train)
svr_rbf_high_c_test_yhat = svr_rbf_high_c.predict(X_test)

# 8. SVR with Nyström RBF Approximation + Linear Kernel (for scalability)
# This involves two steps: transform data then train SVR with linear kernel
nystroem_rbf_transformer = Nystroem(kernel='rbf', gamma=0.2, n_components=100, random_state=42)
X_train_transformed = nystroem_rbf_transformer.fit_transform(X_train)
X_test_transformed = nystroem_rbf_transformer.transform(X_test)

svr_nystrom_rbf = SVR(kernel='linear', C=1.0) # Apply SVR linear to the transformed features
svr_nystrom_rbf.fit(X_train_transformed, y_train)
svr_nystrom_rbf_test_yhat = svr_nystrom_rbf.predict(X_test_transformed)


# --- Evaluation ---

print(cl('EXPLAINED VARIANCE SCORE:', attrs = ['bold']))
print('-------------------------------------------------------------------------------')
print(cl('OLS model:', attrs = ['bold']))
print(cl('Test - RMSE is {:.4f}'.format(np.sqrt(metrics.mean_squared_error(y_test,ols_test_yhat))), attrs = ['bold']))

print('-------------------------------------------------------------------------------')
print(cl('Ridge model:', attrs = ['bold']))
print(cl('Test - RMSE is {:.4f}'.format(np.sqrt(metrics.mean_squared_error(y_test,ridge_test_yhat))), attrs = ['bold']))

print('-------------------------------------------------------------------------------')
print(cl('Lasso model:', attrs = ['bold']))
print(cl('Test - RMSE is {:.4f}'.format(np.sqrt(metrics.mean_squared_error(y_test,lasso_test_yhat))), attrs = ['bold']))

print('-------------------------------------------------------------------------------')
print(cl('Bayesian model:', attrs = ['bold']))
print(cl('Test - RMSE is {:.4f}'.format(np.sqrt(metrics.mean_squared_error(y_test,bayesian_test_yhat))), attrs = ['bold']))

print('-------------------------------------------------------------------------------')
print(cl('SVR (Default RBF, C=1.0) model:', attrs = ['bold']))
print(cl('Test - RMSE is {:.4f}'.format(np.sqrt(metrics.mean_squared_error(y_test, svr_default_test_yhat))), attrs = ['bold']))

print('-------------------------------------------------------------------------------')
print(cl('SVR (Linear Kernel, C=0.1 - More Regularization) model:', attrs = ['bold']))
print(cl('Test - RMSE is {:.4f}'.format(np.sqrt(metrics.mean_squared_error(y_test, svr_linear_low_c_test_yhat))), attrs = ['bold']))

print('-------------------------------------------------------------------------------')
print(cl('SVR (Linear Kernel, C=100.0 - Less Regularization) model:', attrs = ['bold']))
print(cl('Test - RMSE is {:.4f}'.format(np.sqrt(metrics.mean_squared_error(y_test, svr_linear_high_c_test_yhat))), attrs = ['bold']))

print('-------------------------------------------------------------------------------')
print(cl('SVR (RBF Kernel, C=0.1 - More Regularization) model:', attrs = ['bold']))
print(cl('Test - RMSE is {:.4f}'.format(np.sqrt(metrics.mean_squared_error(y_test, svr_rbf_low_c_test_yhat))), attrs = ['bold']))

print('-------------------------------------------------------------------------------')
print(cl('SVR (RBF Kernel, C=100.0 - Less Regularization) model:', attrs = ['bold']))
print(cl('Test - RMSE is {:.4f}'.format(np.sqrt(metrics.mean_squared_error(y_test, svr_rbf_high_c_test_yhat))), attrs = ['bold']))

print('-------------------------------------------------------------------------------')
print(cl('SVR (Nyström RBF Approx. + Linear Kernel) model:', attrs = ['bold']))
print(cl('Test - RMSE is {:.4f}'.format(np.sqrt(metrics.mean_squared_error(y_test, svr_nystrom_rbf_test_yhat))), attrs = ['bold']))


--- SVR Models ---


D:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.543e+05, tolerance: 2.172e+03
  model = cd_fast.enet_coordinate_descent(


EXPLAINED VARIANCE SCORE:
-------------------------------------------------------------------------------
OLS model:
Test - RMSE is 136.0598
-------------------------------------------------------------------------------
Ridge model:
Test - RMSE is 136.3652
-------------------------------------------------------------------------------
Lasso model:
Test - RMSE is 136.4486
-------------------------------------------------------------------------------
Bayesian model:
Test - RMSE is 118.9773
-------------------------------------------------------------------------------
SVR (Default RBF, C=1.0) model:
Test - RMSE is 115.0731
-------------------------------------------------------------------------------
SVR (Linear Kernel, C=0.1 - More Regularization) model:
Test - RMSE is 125.8027
-------------------------------------------------------------------------------
SVR (Linear Kernel, C=100.0 - Less Regularization) model:
Test - RMSE is 137.6789
-----------------------------------------------

Vamos a desglosar esos parámetros clave de SVR y la aproximación de Nystroem.

---

## **Parámetros Clave de SVR**

### 1. `C` (float, default=1.0) - Parámetro de Regularización

* **¿Qué es?** Controla la **fuerza de la regularización**.
* **Relación Inversa:** La fuerza de la regularización es **inversamente proporcional a `C`**.
    * **`C` pequeño (ej., 0.1):** **Mucha regularización**. El modelo se "preocupa" mucho por los errores (penaliza fuertemente los puntos fuera del tubo $\epsilon$). Esto lleva a un modelo más simple, que generaliza mejor y reduce el overfitting, pero puede tener un error de entrenamiento más alto.
    * **`C` grande (ej., 100):** **Poca regularización**. El modelo se "preocupa" menos por los errores y permite que se ajusten más a los datos de entrenamiento. Esto puede llevar a un overfitting si es demasiado grande, ya que el modelo se vuelve muy sensible a cada punto de entrenamiento.
* **Penalización:** La penalización es una **L2 cuadrada** (como en Ridge Regression), lo que significa que tiende a mantener los pesos pequeños, pero no los fuerza a cero.
* **Siempre Positivo:** `C` debe ser estrictamente positivo.

### 2. `epsilon` (float, default=0.1) - Margen de Tolerancia del Tubo

* **¿Qué es?** Define el **tamaño del "tubo"** alrededor de la función de regresión predicha.
* **No Penalización:** Los puntos de entrenamiento que caen **dentro de este tubo** (a una distancia $\epsilon$ o menos de la predicción real) **no contribuyen a la función de pérdida** y, por lo tanto, no se les aplica ninguna penalización.
* **Robustez a Outliers:** Hace que SVR sea **robusto a los outliers**. Si un outlier está dentro del tubo (o no muy lejos), su impacto en el modelo es limitado.
* **Mayor $\epsilon$:** Significa un tubo más ancho. Más puntos caerán dentro del tubo, lo que resulta en un modelo más simple con menos vectores de soporte. Puede aumentar el sesgo.
* **Menor $\epsilon$:** Significa un tubo más estrecho. Más puntos caerán fuera o en el borde del tubo, lo que lleva a un modelo más complejo con más vectores de soporte. Puede reducir el sesgo pero aumentar la varianza.
* **Siempre No Negativo:** `epsilon` debe ser no negativo.

### 3. `gamma` ({'scale', 'auto'} o float, default='scale') - Coeficiente del Kernel

* **¿Qué es?** Coeficiente del kernel para los kernels **'rbf'**, **'poly'** y **'sigmoid'**. Define la **influencia de una sola muestra de entrenamiento** en el modelo.
* **Controla el "Alcance":**
    * **`gamma` pequeño:** Los puntos de datos tienen una **influencia más amplia**. La frontera de decisión (o la superficie de regresión) será más suave y menos "quebrada". Puede llevar a **underfitting**.
    * **`gamma` grande:** Los puntos de datos tienen una **influencia más localizada**. La frontera de decisión será más compleja, intentando ajustarse a cada punto individualmente. Puede llevar a **overfitting**.
* **Valores por defecto (sklearn):**
    * **`'scale'` (predeterminado):** `gamma` se calcula como `1 / (n_features * X.var())`. Es una heurística que escala `gamma` con la varianza de las características, lo que a menudo funciona bien.
    * **`'auto'`:** `gamma` se calcula como `1 / n_features`. Este valor ignora la varianza de los datos.
* **Float:** Si se pasa un número flotante, debe ser no negativo.
* **Impacto:** Es uno de los hiperparámetros más críticos para los kernels no lineales, especialmente RBF, y su ajuste (tuning) es crucial para el rendimiento.

---

## **Aproximación de Nystroem**

### `Nystroem(n_components=100)`

* **¿Qué es?** `Nystroem` no es un parámetro de SVR directamente, sino una **clase separada** de `sklearn.kernel_approximation` (aproximación de kernels).
* **Propósito:** Se utiliza para **acelerar** el entrenamiento de modelos basados en kernels (como SVR con RBF) en **grandes conjuntos de datos**.
* **Cómo funciona:** En lugar de calcular la matriz del kernel completa (que puede ser muy grande y costosa), `Nystroem` **aproxima** el kernel seleccionando aleatoriamente un subconjunto de `n_components` de los datos de entrenamiento para formar un conjunto de "prototipos" o "bases". Luego, transforma los datos originales en un nuevo espacio de características de menor dimensión utilizando la similitud con estos prototipos.
* **`n_components=100`:**
    * Este parámetro especifica el **número de componentes (prototipos)** a utilizar para la aproximación.
    * Un número mayor de componentes (`n_components`) generalmente conduce a una **mejor aproximación** del kernel original, pero también aumenta el tiempo de cálculo.
    * Un número menor reduce el tiempo de cálculo, pero la aproximación puede ser menos precisa.
    * Es un parámetro crucial para equilibrar la **precisión** y la **eficiencia computacional** cuando se usa Nystroem.
* **Uso:** Primero se transforma el `X_train` y `X_test` con `Nystroem`, y luego se entrena un SVR (típicamente con `kernel='linear'`) sobre los datos transformados. Esto simula el efecto de un kernel RBF de forma más escalable.

---

Estos parámetros te dan un control muy fino sobre el comportamiento de tu modelo SVR, permitiéndote ajustarlo para encontrar el equilibrio óptimo entre el ajuste a los datos de entrenamiento y la capacidad de generalización a datos nuevos.

## Seeking for best C and Epsilon on Standardized Data - BRUTE FORCE

In [65]:
best_R_square = np.inf
best_C = None
best_epsilon = None
for c in range (1,100):
    for e in range (1,10):
        C=c**2/2 
        epsilon=e/10
        svr = SVR(C=C,epsilon=epsilon)
        svr.fit(X_train, y_train)
        #bayesian_yhat = bayesian.predict(X_test)
        svr_test_yhat = svr.predict(X_test)
        svr_train_yhat = svr.predict(X_train)
        rsquared = np.sqrt(metrics.mean_squared_error(y_test, svr_test_yhat))
        if rsquared < best_R_square:
            best_R_square = rsquared
            best_C = C
            best_epsilon = epsilon            
            print('-------------------------------------------------------------------------------')        
            print(cl('SVR model Test C={} epsilon={} - R^2 model is {}'.format(C,epsilon,rsquared), attrs = ['bold']))

            
print(cl('BEST SVR model Test C={} epsilon={} - R^2 model is {}'.format(best_C,best_epsilon,best_R_square), attrs = ['bold']))            

-------------------------------------------------------------------------------
SVR model Test C=0.5 epsilon=0.1 - R^2 model is 115.44513732799967
-------------------------------------------------------------------------------
SVR model Test C=0.5 epsilon=0.2 - R^2 model is 115.44466327521863
-------------------------------------------------------------------------------
SVR model Test C=0.5 epsilon=0.3 - R^2 model is 115.44300741643985
-------------------------------------------------------------------------------
SVR model Test C=0.5 epsilon=0.4 - R^2 model is 115.44207315985814
-------------------------------------------------------------------------------
SVR model Test C=2.0 epsilon=0.1 - R^2 model is 114.77856703318176
-------------------------------------------------------------------------------
SVR model Test C=2.0 epsilon=0.2 - R^2 model is 114.7757146757467
-------------------------------------------------------------------------------
SVR model Test C=2.0 epsilon=0.3 - R^2 

In [66]:
print(cl('BEST SVR model Test C={} epsilon={} - R^2 model is {}'.format(best_C,best_epsilon,best_R_square), attrs = ['bold']))            

BEST SVR model Test C=12.5 epsilon=0.1 - R^2 model is 114.21085631460248


# But what is SVR?

reference: https://towardsdatascience.com/an-introduction-to-support-vector-regression-svr-a3ebc1672c2

SVR gives us the flexibility to define how much error is acceptable in our model and will find an appropriate line (or hyperplane in higher dimensions) to fit the data.


In contrast to OLS, the objective function of SVR is to minimize the coefficients — more specifically, the l2-norm of the coefficient vector — not the squared error. The error term is instead handled in the constraints, where we set the absolute error less than or equal to a specified margin, called the maximum error, ϵ (epsilon). We can tune epsilon to gain the desired accuracy of our model. Our new objective function and constraints are as follows:


## Minimize:



<img src="img/svr1.png" align="center">

### Giving Ourselves some Slack (and another Hyperparameter)
The concept of slack variables is simple: for any value that falls outside of ϵ, we can denote its deviation from the margin as ξ.
We know that these deviations have the potential to exist, but we would still like to minimize them as much as possible. Thus, we can add these deviations to the objective function.

<img src="img/svr11.png" align="center">


## Constraints:
<img src="img/svr2.png" align="center">

### With Slack:
<img src="img/svr22.png" align="center">

## Illustrative Example:
<img src="img/svr3.png" align="center">

### With Slack:
<img src="img/svr33.png" align="center">


We now have an additional hyperparameter, C, that we can tune. As C increases, our tolerance for points outside of ϵ also increases. As C approaches 0, the tolerance approaches 0 and the equation collapses into the simplified (although sometimes infeasible) one.

Let’s set C=1.0 and retrain our model above. The results are plotted below:
<img src="img/svr44.png" align="center">


